In [1]:
import os
import json
import base64
from elasticsearch import Elasticsearch, helpers

from pprint import pprint

client = Elasticsearch(
    hosts=["http://localhost:9200"],
    api_key="SnppODJKVUJEUXJXcENRZDFndTA6LTJQclVnbllUNE9LLTJiUHJqSnBRZw==",
)

resp = client.ping()
print(resp)

def output(data):
    json_data = json.dumps(data.body, indent=4)
    print(json_data)



True


In [6]:
resp = client.inference.put(
    task_type="text_embedding",
    inference_id="chunking-elser-model",
    inference_config={
        "service": "elasticsearch",
        "service_settings": {
            "num_allocations": 1,
            "num_threads": 1,
            "model_id": ".elser_model_2"
        }
    },
)
print(resp)



BadRequestError: BadRequestError(400, 'status_exception', 'Model IDs must be unique. Requested model ID [chunking-elser-model] matches existing model IDs but must not.')

In [7]:
try:
    client.indices.delete(index="chunking-index-elser")
except:
    print('Creating index...')
finally:
    resp = client.indices.create(
        index="chunking-index-elser",
        mappings={
            "properties": {
                "chunking_vectors": {
                    "type": "sparse_vector",
                },
                "attachment.content": {
                    "type": "text",

                }
            }
        },
    )
    print(resp)



Creating index...
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'chunking-index-elser'}


In [8]:
resp = client.ingest.put_pipeline(
    id="attachment",
    description="Extract PDF information",
    processors=[
        {
            "attachment": {
                "field": "data",
                "remove_binary": True
            },
            "inference": {
                "model_id": "chunking-elser-model",
                "input_output": [
                    {
                        "input_field": "attachment.content",
                        "output_field": "chunking_vectors"
                    }
                ]
            }
        }

    ],
)
print(resp)



{'acknowledged': True}


In [12]:
pipeline_id = 'attachment'  # the pipeline we have created before
pdf_dir = 'WAC-BIG'  # the folder with the documents 

# Function to convert PDF file to base64-encoded binary

def convert_pdf_to_base64(file_path):
    with open(file_path, "rb") as file:
        print(f'processing {file_path}...')
        return base64.b64encode(file.read()).decode('utf-8')

# Function to generate actions for the bulk API
def generate_actions(pdf_dir):
    for filename in os.listdir(pdf_dir):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(pdf_dir, filename)
            base64_encoded = convert_pdf_to_base64(file_path)
            yield {
                "_index": "chunking-index-elser",
                "_source": {
                    "data": base64_encoded
                }
            }
       

# Use the helpers.bulk() function to index documents in bulk
helpers.bulk(client, generate_actions(pdf_dir), pipeline=pipeline_id, request_timeout=60*3)

print("Finished indexing PDF documents.")

processing WAC-BIG/WAC 182.pdf...


/var/folders/_p/z4x55khn57z2zn7d0ptxsbg00000gn/T/ipykernel_44398/9482065.py:26: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  helpers.bulk(client, generate_actions(pdf_dir), pipeline=pipeline_id, request_timeout=60*3)


Finished indexing PDF documents.
